In [1]:
import sys
sys.path.append('../..')

In [2]:
import re
import typing
from operator import getitem, setitem
from functools import reduce

from datatypes import datatype, match, compare, case, run, placeholder as _


_0, _1, _2, _3 = map(_.__getitem__, range(4))
a, b, c, x, y, z = map(_.__getattr__, 'abcxyz')
xs = _.xs

In [3]:
code = '''
(define (fib x)
    (if (lt x 2)
        x
        (add (fib (sub x 1))
             (fib (sub x 2)))))

(print (fib 10))
'''

In [4]:
@datatype(expose=locals())
class Token:
    WHITESPACE  : r'\s+'
    PARENTHESIS : r'[\(\)]'

    NAME    : r'[a-zA-Z][a-zA-Z0-9]*'
    NUMBER  : r'\d+'
    BOOLEAN : r'#[tf]'


rules : typing.List[typing.Tuple[typing.Type, re.Pattern]]
rules = [
    (c, re.compile('^' + list(c.__annotations__.values())[0]))
    for c in Token._constructors.values()
]

def tokenize(code : str) -> typing.Iterator[Token]:
    while code:
        match, code = tokenize_one(code)
        yield match

def tokenize_one(code : str) -> typing.Tuple[Token, str]:
    for constructor, regex in rules:
        match = regex.search(code)
        if match:
            end = match.end()
            return constructor(code[:end]), code[end:]
    raise Exception(code)

In [5]:
@datatype(expose=locals())
class Expression:
    Literal : typing.Any

    Variable   : str
    Definition : (str, 'Expression')
    Lambda   : (typing.List['Expression'], 'Expression')

    Pair : ('Expression', 'Expression')

    If : ('Expression', 'Expression', 'Expression')

    Application : ('Expression', typing.List['Expression'])

    Null : ()

        
def iter_pair(cons):
    while not isinstance(cons, Null):
        car, cons = match (cons) [Pair(_.car, _.cons) : (_.car, _.cons)]
        yield car
Pair.__iter__ = iter_pair


def parse(code : str):
    tokens = filter(lambda t: not isinstance(t, WHITESPACE), tokenize(code))
    return parse_tokens(list(tokens))


def parse_tokens(tokens):
    while tokens:
        ast, tokens = parse_one(tokens)
        yield ast


def parse_one(tokens):
    head, *tail = tokens

    return match (head) [
        PARENTHESIS('(') : parse_list << _(tail),

        BOOLEAN('#t') : (Literal(True), tail),
        BOOLEAN('#f') : (Literal(False), tail),

        NUMBER(x) : (Literal << (int << x), tail),
        NAME(x) : (Variable(x), tail),
    ]


def parse_list(tokens):
    head, *tail = tokens

    return match (head) [
        PARENTHESIS(')') : (Null(), tail),
        x : parse_otherwise << _(tokens),
    ]


def parse_otherwise(tokens):
    expr1, rest1 = parse_one(tokens)
    expr2, rest2 = parse_list(rest1)
    return Pair(expr1, expr2), rest2


def analyze(partial):
    return match (partial) [
        Null() : Null(),
        Literal(x) : Literal(x),
        Variable(x) : Variable(x),
        Pair(a, b) : analyze_pair << _(Pair(a, b)),
    ]


def analyze_pair(pair):
    return match (pair) [
        Pair(
            Variable('if'),
            Pair(_.predicate, Pair(_.on_true, Pair(_.on_false, Null()))),
        ):
            If << _(
                analyze << _(_.predicate),
                analyze << _(_.on_true),
                analyze << _(_.on_false),
            ),

        Pair(Variable('lambda'), Pair(_.parameters, Pair(_.body, Null()))):
            Lambda << _(list << _(_.parameters), analyze << _(_.body)),

        Pair(Variable('define'), Pair(_.name, Pair(_.value, Null()))):
            Definition << _(_.name, analyze << _(_.value)),

        Pair(_.operator, _.operands):
            Application << _(
                analyze << _(_.operator),
                list << _(map << _(analyze, _.operands)),
            )
    ]


def parse_analyze(code):
    return map(analyze, parse(code))

In [14]:
def evaluate(code, env):
    return evaluate_commands(parse_analyze(code), env)

def repl(env):
    while True:
        env = evaluate(input('> '), env)

def evaluate_commands(cmds, env):
    for cmd in cmds:
        result, env = evaluate_one(cmd, env)
    return env

def evaluate_one(cmd, env):
    return match (cmd) [
        Literal(x):
            (x, env),

        Variable(x):
            (getitem << _(env, x), env),

        Lambda(_.parameters, _.body):
            evaluate_lambda << _(_.parameters, _.body, env),

        Definition(_.name, _.value):
            evaluate_definition << _(_.name, _.value, env),

        If(_.cond, _.on_true, _.on_false):
            evaluate_if << _(_.cond, _.on_true, _.on_false, env),

        Application(_.f, _.args):
            evaluate_application << _(_.f, _.args, env),

        Null():
            Null(),
    ]


def evaluate_lambda(parameters, body, env):
    names = []
    for parameter in parameters:
        names.append(match (parameter) [Variable(x) : x])

    closure = {**env}
    def _run(*args):
        return evaluate_one(
            body,
            {**args[-1], **closure, **dict(zip(names, args[:-1]))},
        )[0]
    return (_run, env)


def evaluate_definition(name, value, env):
    result, env = evaluate_one(value, env)
    env[match (name) [Variable(x): x]] = result
    return None, env


def evaluate_if(cond, on_true, on_false, env):
    result, env = evaluate_one(cond, env)
    if result:
        return evaluate_one(on_true, env)
    return evaluate_one(on_false, env)


def evaluate_application(f, args, env):
    result_args = []
    for arg in args:
        result, env = evaluate_one(arg, env)
        result_args.append(result)
    f, env = evaluate_one(f, env)

    return f(*result_args, env), env

In [15]:
stdlib = {
    'print' : lambda a, _: print(a),
    'add'   : lambda a, b, _: a + b,
    'sub'   : lambda a, b, _: a - b,
    'lt'    : lambda a, b, _: a < b,
}

In [17]:
evaluate('''
(define fib (lambda (x)
    (if (lt x 2)
        x
        (add (fib (sub x 1)) (fib (sub x 2))))))

(print (fib 10))
''', stdlib);

55


In [ ]:
repl(stdlib)

>  (define x (add 1 2))
>  (print x)


3
